In [1]:
import kagglehub

path = kagglehub.dataset_download("gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged/versions/1


In [7]:
!pip install datasets yfinance

  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 101.0 MB/s eta 0:00:00
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=330783ad5b2cede27161f6f57fc3aa56c4f294e0f2cd1975d66a5da38396edf3
  Stored in directory: /root/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.2-cp312-cp312-linux_x86_64.whl size=936678 sha256=33

In [3]:
ls /root/.cache/kagglehub/datasets/gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged/versions/1

mbsa.csv


In [4]:
from datasets import load_dataset

ds = load_dataset("edaschau/bitcoin_news")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/695 [00:00<?, ?B/s]

BTC_match_text.csv:   0%|          | 0.00/495M [00:00<?, ?B/s]

BTC_match_title.csv:   0%|          | 0.00/116M [00:00<?, ?B/s]

BTC_yahoo.csv:   0%|          | 0.00/396M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/210832 [00:00<?, ? examples/s]

In [10]:
import pandas as pd
from datasets import load_dataset
import os
import numpy as np
from datetime import timedelta
from tqdm.notebook import tqdm
import yfinance as yf

print("Downloading price data for Bitcoin, Gold, and Oil...")
data = yf.download('BTC-USD GC=F CL=F', start='2024-5-31', end='2023-01-31', progress=False)
df_prices = pd.DataFrame(data['Close']['BTC-USD']).rename(columns={'BTC-USD': 'Close'})
df_gold = pd.DataFrame(data['Close']['GC=F']).rename(columns={'GC=F': 'Close'})
df_oil = pd.DataFrame(data['Close']['CL=F']).rename(columns={'CL=F': 'Close'})
df_prices.index = df_prices.index.tz_localize('UTC')
df_gold.index = df_gold.index.tz_localize('UTC')
df_oil.index = df_oil.index.tz_localize('UTC')
print("✅ Price data loaded.")

print("\nLoading LLM-Augmented Bitcoin dataset from Hugging Face...")
try:
    ds_llm = load_dataset("danilocorsi/LLMs-Sentiment-Augmented-Bitcoin-Dataset")
    df_llm_metrics = ds_llm['train'].to_pandas()
    df_llm_metrics['datetime'] = pd.to_datetime(df_llm_metrics['timestamp'], errors='coerce', utc=True)
    df_llm_metrics.dropna(subset=['datetime'], inplace=True)
    df_llm_metrics.set_index('datetime', inplace=True)
    df_llm_metrics.sort_index(inplace=True)
    print("✅ LLM-Augmented metrics loaded.")
except Exception as e:
    print(f"❌ Could not load LLM-Augmented dataset. Error: {e}")
    df_llm_metrics = pd.DataFrame()


print("\nLoading tweets dataset...")
try:
    path
    kaggle_csv_path = os.path.join(path, "mbsa.csv")
    df_tweets = pd.read_csv(kaggle_csv_path, usecols=['Date', 'text'], nrows=100000000)
    df_tweets['datetime'] = pd.to_datetime(df_tweets['Date'], errors='coerce', utc=True)
    df_tweets.dropna(subset=['datetime'], inplace=True)
    df_tweets.set_index('datetime', inplace=True)
    df_tweets.sort_index(inplace=True)
    print("✅ Tweets loaded.")
except NameError:
    print("❌ Error: The 'path' variable is not defined.")
    df_tweets = pd.DataFrame()


print("\nLoading news dataset...")
ds_news = load_dataset("edaschau/bitcoin_news")
df_news = ds_news['train'].to_pandas()
df_news['datetime'] = pd.to_datetime(df_news['date_time'], errors='coerce', utc=True)
df_news.dropna(subset=['datetime'], inplace=True)
df_news.set_index('datetime', inplace=True)
df_news.sort_index(inplace=True)
print("✅ News loaded.")



def get_value_for_day(df, day_str, col_name):
    if df.empty or day_str not in df.index:
        return np.nan
    try:
        value = df.loc[day_str, col_name]
        if not pd.api.types.is_scalar(value): value = value.iloc[0]
        return value
    except (KeyError, IndexError, AttributeError):
        return np.nan

daily_dates = pd.date_range(start='2018-02-01', end='2024-5-31', freq='D', tz='UTC')
daily_data_list = []

print(f"\n⚙️  Processing {len(daily_dates)} days to build the final dataset...")
for day in tqdm(daily_dates, desc="Aggregating Daily Data"):
    day_str = day.strftime('%Y-%m-%d')

    news_text_list = df_news.loc[day_str].head(30)['article_text'].tolist() if day_str in df_news.index else []
    tweets_text_list = df_tweets.loc[day_str].head(30)['text'].tolist() if day_str in df_tweets.index else []
    past_prices_df = df_prices[(df_prices.index >= (day - timedelta(days=60))) & (df_prices.index < day)]
    past_60_day_prices = past_prices_df['Close'].tolist()
    future_prices_df = df_prices[(df_prices.index > day) & (df_prices.index <= (day + timedelta(days=10)))]
    future_10_day_prices = future_prices_df['Close'].tolist()
    gold_price = get_value_for_day(df_gold, day_str, 'Close')
    oil_price = get_value_for_day(df_oil, day_str, 'Close')
    past_news_df = df_news[(df_news.index >= (day - timedelta(days=60))) & (df_news.index < day)]
    random_article_text = None
    if not past_news_df.empty:
        random_article_text = past_news_df.sample(n=1)['article_text'].iloc[0]

    hash_rate = get_value_for_day(df_llm_metrics, day_str, 'hash-rate')
    difficulty = get_value_for_day(df_llm_metrics, day_str, 'difficulty')
    transactions = get_value_for_day(df_llm_metrics, day_str, 'n-transactions')
    unique_addresses = get_value_for_day(df_llm_metrics, day_str, 'n-unique-addresses')
    fng_index = get_value_for_day(df_llm_metrics, day_str, 'fng_value')
    cbbi_index = get_value_for_day(df_llm_metrics, day_str, 'cbbi_value')
    llm_sentiment = get_value_for_day(df_llm_metrics, day_str, 'sentiment_class')
    llm_action = get_value_for_day(df_llm_metrics, day_str, 'action_class')
    market_cap = get_value_for_day(df_llm_metrics, day_str, 'market-cap')
    total_bitcoins = get_value_for_day(df_llm_metrics, day_str, 'total-bitcoins')
    estimated_tx_volume = get_value_for_day(df_llm_metrics, day_str, 'estimated-transaction-volume-usd')
    text_cointelegraph = get_value_for_day(df_llm_metrics, day_str, 'cointelegraph')
    text_bitcoin_news = get_value_for_day(df_llm_metrics, day_str, 'bitcoin_news')
    text_reddit = get_value_for_day(df_llm_metrics, day_str, 'reddit')

    daily_data_list.append({
        'date': day.date(),
        'daily_news_full_text': news_text_list,
        'daily_tweets': tweets_text_list,
        'contextual_past_article': random_article_text,
        'gold_close_price': gold_price,
        'oil_close_price': oil_price,
        'btc_price_history_60d': past_60_day_prices,
        'btc_price_target_10d': future_10_day_prices,
        'btc_hash_rate': hash_rate,
        'btc_difficulty': difficulty,
        'btc_transactions': transactions,
        'btc_unique_addresses': unique_addresses,
        'fear_and_greed_index': fng_index,
        'cbbi_index': cbbi_index,
        'llm_sentiment_class': llm_sentiment,
        'btc_market_cap': market_cap,
        'btc_total_supply': total_bitcoins,
        'btc_estimated_tx_volume_usd': estimated_tx_volume,
        'text_cointelegraph': text_cointelegraph,
        'text_bitcoin_news': text_bitcoin_news,
        'text_reddit_posts': text_reddit,
    })


/tmp/ipython-input-701122676.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('BTC-USD GC=F CL=F', start='2024-5-31', end='2023-01-31', progress=False)
ERROR:yfinance:
3 Failed downloads:
ERROR:yfinance:['BTC-USD']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-5-31 -> 2023-01-31) (Yahoo error = "Invalid input - start date cannot be after end date. startDate = 1717113600, endDate = 1675123200")')
ERROR:yfinance:['CL=F', 'GC=F']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-5-31 -> 2023-01-31) (Yahoo error = "Invalid input - start date cannot be after end date. startDate = 1717128000, endDate = 1675141200")')


✅ Price data loaded.

Loading LLM-Augmented Bitcoin dataset from Hugging Face...
✅ LLM-Augmented metrics loaded.

Loading tweets dataset...
✅ Tweets loaded.

Loading news dataset...
✅ News loaded.

⚙️  Processing 2312 days to build the final dataset...


Aggregating Daily Data:   0%|          | 0/2312 [00:00<?, ?it/s]

In [12]:


final_dataset = pd.DataFrame(daily_data_list)

print("\n🔄 Filling missing values...")
cols_to_fill_numeric = [
    'gold_close_price', 'oil_close_price', 'btc_hash_rate', 'btc_difficulty',
    'btc_transactions', 'btc_unique_addresses', 'fear_and_greed_index', 'cbbi_index',
    'btc_market_cap', 'btc_total_supply', 'btc_estimated_tx_volume_usd'
]
final_dataset[cols_to_fill_numeric] = final_dataset[cols_to_fill_numeric].ffill()
cols_to_fill_text = [
    'llm_sentiment_class', 'text_cointelegraph',
    'text_bitcoin_news', 'text_reddit_posts'
]
final_dataset[cols_to_fill_text] = final_dataset[cols_to_fill_text].fillna('no_data')
print("✅ Missing values filled.")

print("\n\n✅ Final rich dataset created successfully!")
display(final_dataset.head())


🔄 Filling missing values...
✅ Missing values filled.


✅ Final rich dataset created successfully!


,date,daily_news_full_text,daily_tweets,contextual_past_article,gold_close_price,oil_close_price,btc_price_history_60d,btc_price_target_10d,btc_hash_rate,btc_difficulty,...,btc_unique_addresses,fear_and_greed_index,cbbi_index,llm_sentiment_class,btc_market_cap,btc_total_supply,btc_estimated_tx_volume_usd,text_cointelegraph,text_bitcoin_news,text_reddit_posts
0,2018-02-01,[Tax reform has had a major impact on the curr...,"[Be judicious, buy your bitcoin at https://Bit...",* Apple down on report of tepid iPhone X deman...,NaN,NaN,[],[],2.070395e+07,2.603077e+12,...,591551.0,0.30,0.77,negative,1.972803e+11,16832075.0,2.614261e+09,"[[15052, 'regulated-government-issued-cryptos-...","[[106827, '2018-02-01 23:55:06', 'Senate of Ni...","[['u/Beastly4k', 'OmiseGO primed for 2018! A b..."
1,2018-02-02,[E-commerce veteran and cloud-computing expert...,"[Bitcoin @ £6,109.31 | Ethereum @ £665.00 | Li...","Having successfully defended $16,000 earlier t...",NaN,NaN,[],[],2.070395e+07,2.603077e+12,...,591551.0,0.15,0.76,negative,1.972803e+11,16832075.0,2.614261e+09,no_data,no_data,no_data
2,2018-02-03,[National Oilwell Varco(NYSE: NOV)will report ...,[#ada Ada bull run has begun. Going to $2.00 o...,"By Gertrude Chavez-Dreyfuss, Jemima Kelly and ...",NaN,NaN,[],[],2.044515e+07,2.603077e+12,...,436197.0,0.40,0.75,negative,1.972803e+11,16832075.0,1.469365e+09,"[[15088, 'cryptocurrency-markets-move-back-int...","[[107636, '2018-02-03 23:30:30', 'Bitcoin.com’...","[['u/01-23_45', 'A Theory of Tether', 42, '201..."
3,2018-02-04,[If you file for Social Security benefits at a...,[2018-02-04 00:46:52 Bitcoin price currently a...,General Motors (NYSE: GM) is coming off a stro...,NaN,NaN,[],[],2.044515e+07,2.603077e+12,...,436197.0,0.24,0.75,neutral,1.972803e+11,16832075.0,1.469365e+09,no_data,no_data,no_data
4,2018-02-05,[Bitcoin Cash Falls Again The weekend failed t...,[I have a close family member (naming who name...,"""A stock dividend is something tangible -- it'...",NaN,NaN,[],[],2.393894e+07,2.603077e+12,...,486554.0,0.11,0.71,neutral,1.972803e+11,16832075.0,1.950771e+09,"[[15118, 'traditional-markets-nosedive-followi...","[[108411, '2018-02-05 23:55:27', 'New York Wan...","[['u/Timelapze', 'First commercial of the supe..."


In [ ]:
from datasets import load_dataset
import pandas as pd

# Define the repository ID and the specific file path
repo_id = "danilocorsi/LLMs-Sentiment-Augmented-Bitcoin-Dataset"
file_path = "annotated/merged_daily_mistral-nemo_12b-instruct-2407-q5_K_S_opinion.csv"

# Load the specific CSV file
# We pass the file path in the 'data_files' argument.
# The key 'train' is the name we give to this data split.
ds = load_dataset(repo_id, data_files={'train': file_path})

print("✅ File loaded successfully!")
print(ds)

In [ ]:
len(df_llm_metrics["reddit"])

25718

In [ ]:
!pip install yfinance

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 105.6 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=84f5bdb2fa4f3df461595e22e303916fe7d69c3879f6e949d42ce3456f19cb51
  Stored in directory: /root/.cache/pip/wheels/42/d6/84/bf57a755f4569494cd00de4bb46ef064874823f4d19c82e960
  Created wheel for peewee: filename=peewee-3.18.2-cp311-cp311-linux_x86_64.whl size=886654 sha256=86e7c38be3c0b7bbcd492482aa756de57b8205888e28404c737117c316022a78
  Stored in directory: /root/.cache/pip/wheels/28/84/61/7

In [ ]:
final_dataset["daily_news_full_text"][40]

['Major U.S. and European indexes struggled on Monday as Wall Street slogs through an earnings seasons that has largely disappointed. CNBC "Fast Money" trader Brian Kelly said that looking for growth in the current market is "dangerous." Opportunities can still be found, however, and CNBC\'s "Fast Money" traders discussed the sectors they believe hold the best growth prospects. Economic trends point to strong growth for companies in the housing sector, trader Pete Najarian said. Some home builders have "plenty of room to the upside still," he argued. "I look at the way the economy is improving, I look at the lower rates and I look at the cash positions of some of these," Najarian said. Read More Why HK property won\'t bring down its economy He eyed home construction materials producer Masco (NYSE: MAS) in particular. It traded to about $26 per share on Monday. Trader Karen Finerman looked to a newer investment-drones. She has yet to take on any exposure to the sector. However, she says

In [15]:
import pandas as pd
from datasets import Dataset


print("Starting transformation with updated instruction format...")

llm_finetuning_data = []

for index, row in final_dataset.iterrows():

    price_history_str = ", ".join([f"{p:.2f}" for p in row['btc_price_history_60d']])
    news_str = (str(row['daily_news_full_text'][:10000]) + '...') if row['daily_news_full_text'] else "No specific news available."

    instruction = (
        "You are an expert financial analyst. Your primary task is to predict the next 10 days of Bitcoin prices. "
        "Analyze the provided news and historical price data to make your forecast.\n\n"
        f"**Today's Key News:**\n{news_str}\n\n"
        f"**Last 60 Days of Bitcoin's Closing Prices:**\n[{price_history_str}]\n\n"
        "Use the additional daily data provided in the input below for crucial context."
    )


    gold_price = f"${row['gold_close_price']:.2f}" if pd.notna(row['gold_close_price']) else "N/A"
    oil_price = f"${row['oil_close_price']:.2f}" if pd.notna(row['oil_close_price']) else "N/A"
    market_cap = f"${row['btc_market_cap']:,.0f}" if pd.notna(row['btc_market_cap']) else "N/A"
    fear_greed = f"{row['fear_and_greed_index']:.2f}" if pd.notna(row['fear_and_greed_index']) else "N/A"

    input_text = f"""
Daily Context for Date: {row['date']}

**Financial & Commodity Data:**
- Gold Closing Price: {gold_price}
- Crude Oil Closing Price: {oil_price}

**Bitcoin Market & On-Chain Metrics:**
- Market Capitalization: {market_cap}
- Hash Rate: {row['btc_hash_rate']}
- Transaction Count: {row['btc_transactions']}
- Unique Addresses: {row['btc_unique_addresses']}

**Social & AI Sentiment:**
- Fear & Greed Index: {fear_greed}

**Other Textual Data:**
- Daily Tweets (Sample): {str(row['daily_tweets'][:10000]) + '...' if row['daily_tweets'] else 'N/A'}
- Contextual Past News Article: {str(row['contextual_past_article'][:10000]) + '...' if row['contextual_past_article'] else 'N/A'}
- Reddit Posts (Sample): {str(row['text_reddit_posts'][:10000]) + '...' if row['text_reddit_posts'] else 'N/A'}

Based on the news and historical prices provided in the instruction and this contextual data, predict the next 10 days of Bitcoin's closing price.
"""

    output_text = ", ".join([f"{p:.2f}" for p in row['btc_price_target_10d']])

    llm_finetuning_data.append({
        'instruction': instruction.strip(),
        'input': input_text.strip(),
        'output': output_text
    })


df_finetuning_full = pd.DataFrame(llm_finetuning_data)
hf_finetuning_dataset_full = Dataset.from_pandas(df_finetuning_full)

print("\n✅ Dataset transformation complete with the new instruction format!")

Starting transformation with updated instruction format...

✅ Dataset transformation complete with the new instruction format!


In [ ]:
df_finetuning_full = pd.DataFrame(llm_finetuning_data)
hf_finetuning_dataset_full = Dataset.from_pandas(df_finetuning_full)


In [20]:
df_finetuning_full["input"][1816]

'Daily Context for Date: 2023-01-22\n\n**Financial & Commodity Data:**\n- Gold Closing Price: N/A\n- Crude Oil Closing Price: N/A\n\n**Bitcoin Market & On-Chain Metrics:**\n- Market Capitalization: $440,463,659,625\n- Hash Rate: 282163369.5939428\n- Transaction Count: 250760.0\n- Unique Addresses: 610248.0\n\n**Social & AI Sentiment:**\n- Fear & Greed Index: 0.53\n\n**Other Textual Data:**\n- Daily Tweets (Sample): N/A\n- Contextual Past News Article: Amid growing interest in self-custody solutions, Bitcoin-centric startup Foundation Devicesannounced todaythat it has closed a $7 million seed funding round. The Boston-based companysaidthat it will use the cash to continue developing its crypto “digital sovereignty” products, which include both hardware and software wallets.\nBlockchain investment firm Polychain Capital led the round, with Greenfield Capital, Lightning Ventures, Unpopular Ventures, Warburg Serres, and Bolt also participating.\nFoundation’s flagship product, Passport, is 

In [24]:
# 1. Define the name for your dataset on the Hub
# ❗️ IMPORTANT: Replace "your-username" with your actual Hugging Face username.
repo_id = "tahamajs/bitcoin-llm-finetuning-dataset_longer"

# 2. Push the dataset to the Hub
print(f"Uploading dataset to '{repo_id}'...")
hf_finetuning_dataset_full.push_to_hub(repo_id)

print(f"\n🚀 Success! View your dataset at: https://huggingface.co/datasets/{repo_id}")

Uploading dataset to 'tahamajs/bitcoin-llm-finetuning-dataset'...


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   0%|          |  533kB /  112MB            

README.md: 0.00B [00:00, ?B/s]


🚀 Success! View your dataset at: https://huggingface.co/datasets/tahamajs/bitcoin-llm-finetuning-dataset


In [23]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `LLM` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache